In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv
import traceback
import datetime
import os

In [2]:
# 保管先フォルダの名称
OUTPUT_DIR = 'livedoor_news'
# 話題のニュース（ライブドアニュース）のURL
URL = 'https://news.livedoor.com/social_reaction/%s/'
# ニュースID抽出用の正規表現（名前付きグループ）
REG_URL = r'(?P<L1>(https?://[^/]+/))(?P<L2>([^/]+))/(?P<L3>([^/]+))/(?P<L4>([^/]+))/'
# ニュースアーカイブの遡及可能な日数
DAYS_AGO_MAX = 570
# UserAgent
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'

In [3]:
def create_folder():
    if not os.path.exists(OUTPUT_DIR):
        # フォルダが存在しなければ、フォルダを作成
        os.makedirs(OUTPUT_DIR)

In [4]:
def get_links(delta=None):
    # 基準日を実行時点で初期化
    target = datetime.date.today()
    #print(target.strftime('%Y-%m-%d'))
    if delta in list(range(-DAYS_AGO_MAX, 1)):
        # delta日前を求めるtimedeltaを作成
        td = datetime.timedelta(days=delta)
        # 基準日のdelta日前を計算して取得
        target = target + td
    #print(target.strftime('%Y-%m-%d'))
    try:
        print(URL %target.strftime('%Y-%m-%d'))
        # HTTPリクエストヘッダーにUser Agentを設定
        req = Request(URL %target.strftime('%Y-%m-%d'), data=None, headers={'User-Agent': USER_AGENT})
        with urlopen(req) as res:
            # HTMLドキュメントからBeautifulSoupを初期化
            soup = BeautifulSoup(res.read().decode('euc_jp', 'ignore'), 'html.parser')
            # 話題のニュース部分を検索し、全てを取得
            soupNewsTopic = soup.find('ol', class_='socialReactionList').find_all('li')

            articles = []
            for idx, soupNews in enumerate(soupNewsTopic):
                # ICON(FacebookとTwitter)を除外
                if len(soupNews.contents) == 0 or soupNews.contents[0].name == 'i':
                    continue
                # 詳細ページURLをHTMLタグの属性から抽出
                #url = soupNews.a.get('href')
                url = soupNews.find('a')['href']
                # NewsBodyを検索し取得
                soupNewsBody = soupNews.find('div', class_='socialReactionBody')
                # NewsBodyから各種データを抽出
                article = {
                    'url': url,
                    # ニュースIDを詳細ページURLから抽出
                    'id': re.search(REG_URL, url).groupdict()['L4'],
                    # タイトル／サマリをHTMLタグの本文から抽出
                    'title': soupNewsBody.find('h3', class_='socialReactionTtl').text,
                    'summary': soupNewsBody.find('p', class_='socialReactionDesc').text
                }
                articles.append(article)
                #print('%2d: %s' %(idx + 1, soupNewsBody.find('h3', class_='socialReactionTtl').text))

            df = pd.DataFrame(articles)
            # DataFrameからCSVファイルを生成
            # encoding='sjis'だとJupyterLab（CSVTable）上で表示不可なことに注意
            df.to_csv('%s/livedoor_news_topic_%s.csv' %(OUTPUT_DIR, target.strftime('%Y-%m-%d')), encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)
    except Exception as e:
        # エラー概要
        print('Exception: ', e)
        print('=====')
        # エラー詳細（スタックトレース）
        print(traceback.format_exc().rstrip())
        print('=====')

In [6]:
import time

create_folder()
for i in range(-6, 1): # 直近、実行当日含めた一週間を取得する場合
    get_links(i)
    time.sleep(5)

https://news.livedoor.com/social_reaction/2021-10-01/
https://news.livedoor.com/social_reaction/2021-10-02/
https://news.livedoor.com/social_reaction/2021-10-03/
https://news.livedoor.com/social_reaction/2021-10-04/
https://news.livedoor.com/social_reaction/2021-10-05/
https://news.livedoor.com/social_reaction/2021-10-06/
https://news.livedoor.com/social_reaction/2021-10-07/


In [8]:
# CSVファイルからDataFrameを生成
df = pd.read_csv('livedoor_news/livedoor_news_topic_2021-10-07.csv', encoding='utf-8')
df.head()

,url,id,title,summary
0,https://news.livedoor.com/topics/detail/209881...,20988126,インフルエンザ「今季は大流行の恐れ」 日本感染症学会は接種を呼びかけ,昨季はコロナ禍で感染予防対策が徹底され、インフルエンザ患者が激減した
1,https://news.livedoor.com/article/detail/20986...,20986713,魚料理に合うのはどれ？自分好みのお米が分かる「お米チャート」,揚げ物にはもっちり硬め？好みの品種も見つけられる「お米チャート」2021/1…
2,https://news.livedoor.com/topics/detail/209874...,20987421,23年逃走中の米最重要手配犯が大リーグの試合観戦か 映像に似た人物発見,よく似た人物が2016年8月に大リーグの試合を観戦していたと分かった
3,https://news.livedoor.com/article/detail/20988...,20988253,大型鳥類エミューが観光牧場から脱走、１１羽捕獲できず,熊本県菊池市の「熊本県菊池エミュー観光牧場」で７日朝、飼育されている…
4,https://news.livedoor.com/article/detail/20988...,20988193,モルカーのNintendo Switch用ゲーム発売決定 最大4人で対戦可能,株式会社バンダイナムコエンターテインメントは、Nintendo Switch用ソフ…


In [10]:
import time

create_folder()
for i in range(-DAYS_AGO_MAX+20, -DAYS_AGO_MAX+30): # 遡及可能な最も古い日付まで全てを取得する場合
    get_links(i)
    time.sleep(5)

https://news.livedoor.com/social_reaction/2020-04-05/
https://news.livedoor.com/social_reaction/2020-04-06/
https://news.livedoor.com/social_reaction/2020-04-07/
https://news.livedoor.com/social_reaction/2020-04-08/
https://news.livedoor.com/social_reaction/2020-04-09/
https://news.livedoor.com/social_reaction/2020-04-10/
https://news.livedoor.com/social_reaction/2020-04-11/
https://news.livedoor.com/social_reaction/2020-04-12/
https://news.livedoor.com/social_reaction/2020-04-13/
https://news.livedoor.com/social_reaction/2020-04-14/


In [12]:
# CSVファイルからDataFrameを生成
df = pd.read_csv('livedoor_news/livedoor_news_topic_2020-04-09.csv', encoding='utf-8')
df.head()

,url,id,title,summary
0,https://news.livedoor.com/article/detail/18096...,18096864,全長120メートルの「超巨大なヒモ」のような生物が発見される、シロナガスクジラを抜いて世界最...,既知の生物の中で最大の動物はシロナガスクジラだと考えられていますが、全…
1,https://news.livedoor.com/article/detail/18095...,18095386,長寿番組『キユーピー3分クッキング』の過去アーカイブ1500作品以上を無料配信,新型コロナウィルス感染拡大防止のための緊急事態宣言発令に伴い、キッズ・…
2,https://news.livedoor.com/topics/detail/180933...,18093398,米でひよこ「パニック買い」急増 自宅での飼育を思い立つ人が相次ぐ,各地で外出禁止令が出る中、販売業者は「パニック買い」と形容
3,https://news.livedoor.com/article/detail/18096...,18096086,内定者に「邪魔」入社前に自殺,就職が内定している企業の人事課長からパワーハラスメントを受け、大学4…
4,https://news.livedoor.com/topics/detail/180943...,18094375,「緊急事態宣言出ているのに営業」腹立てドア破壊したとして男逮捕,「緊急事態宣言が出ているのに営業していて、頭に来た」と話しているという
